In [5]:
import cfscrape
from lxml import etree
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from func_timeout import func_timeout, FunctionTimedOut
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from func_timeout import func_timeout, FunctionTimedOut
import pickle
import requests
from selenium.webdriver.common.action_chains import ActionChains
import math
from calendar import monthrange
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pyperclip 

### Navigator Search

In [466]:
%%time
for i in range(0, names_pool.shape[0]): 
    sleep(randint(1,3))
    browser.implicitly_wait(1)
    name = names_pool['Grantees'].loc[i] #i
    new_name = re.sub(r'( TRUSTEE| LLC| EST| OF| TRUST| INC| LTDA)','', name)
    new_name = re.sub(r'(( \w)+$)','', new_name)
    
    new_name = re.sub(r'( \w )',' ', new_name)
    new_name = re.sub(r'(\, )',' ', new_name)

    names_pool['Searching Name'].loc[i] = new_name

    # input name
    instr_path = "/html/body/div[5]/main/div[1]/form/div/div/div[1]/div/div[1]/div/input"
    # browser.find_element_by_xpath(instr_path).send_keys(new_name)
    #browser.find_element_by_class_name('artdeco-typeahead__input full-width').send_keys(new_name)
    browser.find_element_by_xpath(instr_path).send_keys(new_name)
    
    sleep(0.4)
    browser.find_element_by_xpath(instr_path).send_keys(Keys.ENTER)
    print(i, new_name)

    sleep(0.5)
    actions = ActionChains(browser)
    for _ in range(4):
        actions.send_keys(Keys.PAGE_DOWN).perform()
        sleep(0.1)

    browser.implicitly_wait(1)
    sleep(0.5)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    browser.find_element_by_xpath(instr_path).clear()

    #sleep(1)
    raw_names = soup.findAll('dt', attrs={'class':'result-lockup__name'})
    found_names = []
    for raw_name in raw_names:
        name = re.sub('\n','', raw_name.text)
        name = re.sub('  +|^ | $','', name)
        found_names.append(name)
    if len(found_names) == 0:
        found_names='Not found'
        print('Not found', '\n')
    elif len(found_names) > 5:
        print(', '.join(found_names[:5]), '\n')
    else:
        print(', '.join(found_names), '\n')

    names_pool['Found Names'].loc[i] = found_names
    sleep(1)
    els = soup.findAll('a', attrs={'class':'ember-view'})
    found_links = [] 
    for a in els:
        if '/sales/people/' in a['href'] and a['href'] not in found_links:
            found_links.append(a['href'])
    names_pool['Found Links'].loc[i] = found_links

    # results
    try:
        path = '//*[@id="search-spotlight-tab-ALL"]/span[1]'
        results = browser.find_element_by_xpath(path).text
        names_pool['Found Results'].loc[i] = results
    except:
        names_pool['Found Results'].loc[i] = np.nan

    browser.find_element_by_xpath(instr_path).clear()

    if i%50==0 or i==names_pool.shape[0]-1:
        sleep(randint(10,20))
        print(i,'Saved')
        
        missed_cols = ['Found Names','Found Links']
        print('Missed')
        print(i+1-(names_pool.shape[0]-names_pool[missed_cols].isna().sum()),'\n')
        with open('Precious-Pool-LinkedIn_Pickle.pkl', 'wb') as f:
            pickle.dump(names_pool, f)

### Getting Navigator Profiles Links

In [ ]:
%%time
all_profiles = []
for i in range(5331,5335):#names_pool.shape[0]
    sleep(1)
    
    profiles_list = []
    sales_links = []
    if len(names_pool['Navigator Matched Links'].loc[i])>0:
        
        if len(names_pool['Navigator Matched Links'].loc[i])>10:
            sales_links = names_pool['Navigator Matched Links'].loc[i][:10]
        else:
            sales_links = names_pool['Navigator Matched Links'].loc[i]
        
        for sales_link in sales_links:
            #try:
            url = 'https://www.linkedin.com'+sales_link
            sleep(randint(1,2))
            browser.get(url)

            # click More
            browser.implicitly_wait(0.3)
            #path = '/html/body/div[5]/main/div[1]/div[2]/div/div[2]/div[1]/div[3]/button'
            path=  '/html/body/main/div[1]/div[2]/div/div[2]/div[1]/div[3]/button'


            browser.find_element_by_xpath(path).click()
            #element = WebDriverWait(browser, 6).until(EC.element_to_be_clickable((By.XPATH, path)))
            #element.click()
            #

            #browser.implicitly_wait(0.3)
            path = '/html/body/div[5]/main/div[1]/div[2]/div/div[2]/div[1]/div[3]/div/div/div/div/div[1]/div/ul/li[4]/div'
            browser.find_element_by_xpath(path).click()
            #element = WebDriverWait(browser, 6).until(EC.element_to_be_clickable((By.XPATH, path)))
            #element.click()

            sleep(0.2)
            profile = pyperclip.paste()
            profiles_list.append(profile)
            all_profiles.append(profile)

            #except:
                #print('#', i, 'Except')
                #pass
    else:
        pass

    names_pool['Navigator Profiles'].loc[i] = profiles_list
    print('#', i, 'Match Links Length:', len(sales_links), 'Got Links:', len(names_pool['Navigator Profiles'].loc[i]), 'Profiles', len(profiles_list))
    
    #if i%10==0:
        #print('#', i, 'Match Links Length:', len(sales_links), 'Got Links:', len(profiles_list))
    
    if i%50==0 or i == names_pool.shape[0]-1:
        print('Saved at:', i)
        #print(profiles_list)
        
        sleep(randint(5,10))
        with open('Precious-Pool-LinkedIn_Pickle.pkl', 'wb') as f:
            pickle.dump(names_pool, f)